# Sandbox
This notebook covers the main steps in deploying a model.  Experimentation with features is encouraged.  Can you increase the NDCG reported during model training?

## Setup Client

In [ ]:
from ltr.client import ElasticClient
client = ElasticClient()

## Step 1 - Create a Feature Set

In [ ]:
'''
  TASK:
  Experiment with this featureset, the judgments are all title searches so what features could be useful?
  
  Ideas:
    - Search other fields
    - Phrase matches
    - Fuzzy matches
'''

config = {
   "featureset": {
        "features": [
            {
                "name": "title_match",
                "params": ["keywords"],
                "template": {
                    "match": {
                        "title": "{{keywords}}"
                    }
                }
            }
        ]
    }
}

from ltr import setup
setup(client, index='tmdb', config=config, featureset='sandbox')

## Step 2 - Log Features for Training

In [ ]:
from ltr.log import judgments_to_training_set
trainingSet = judgments_to_training_set(client, 
                                        judgmentInFile='data/title_judgments.txt', 
                                        trainingOutFile='data/sandbox_judgments_train.txt', 
                                        featureSet='sandbox')

## Step 3 - Train a Model

In [ ]:
'''
  TASK:
  Experiment with the leafs and trees variables, how do they affect NGCG?
  Does a high leaf value increase your NDCG?  What could be the potential downfalls?
'''
from ltr import train
trainLog  = train(client,
                  index='tmdb',
                  trainingInFile='data/sandbox_judgments_train.txt',
                  metric2t='NDCG@10',
                  leafs=20,
                  trees=20,
                  featureSet='sandbox',
                  modelName='sandbox')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
for roundDcg in trainLog.rounds:
    print(roundDcg)
    
print("Train NDCG@10 %s" % trainLog.rounds[-1])

## Search

In [ ]:
from ltr import search
search(client, "rambo", modelName='sandbox')